# Imports

In [6]:
import torch
import pandas as pd
from scipy.spatial.distance import cosine
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

In [7]:
training_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device=training_device

In [11]:
df=pd.read_csv("Data/cf_train_no_noise.csv")

# Custom Dataloader

In [20]:
class CustomDataset:
    def __init__(self,dataframe,batch_size,device =training_device,shuffle=False):
        self.df=dataframe
        self.batch_size=batch_size
        self.columns_to_drop=['row_num','day','era','target_10_val','target_5_val','sigma','day_no']
        self.X = self.df.drop(self.columns_to_drop, axis=1)
        self.y=self.df['target_10_val']
        self.device=device
        self.shuffle=shuffle

    def generate_batches_with_labels(self,start_idx,end_idx):
        data=self.X.iloc[start_idx:end_idx]
        labels=self.y.iloc[start_idx:end_idx]
        dataset =  torch.utils.data.TensorDataset(torch.tensor(data.values),torch.tensor(labels.values))
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=self.shuffle)
        # data_unseen = self.X.iloc[max(0,idx-9):idx+1]
        # labels_unseen=self.y.iloc[max(0,idx-9):idx+1]
        # data_unseen,labels_unseen = torch.tensor(data_unseen.values).to(self.device),torch.tensor(labels_unseen.values).to(self.device)
        return dataloader
    
        

In [22]:
customDS = CustomDataset(df,64,shuffle = False)
# supervised_dl, unsupervised_data = customDS.generate_batches_with_labels(77)

# Model

In [4]:
class MyMLP(torch.nn.Module):
    def __init__ (self, layers,activation = torch.nn.ReLU(), dropout = 0.5):
        super().__init__()
        linear = [torch.nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.layers = []
        for i in range(len(linear)):
            if (i == len(linear)-1):
                self.layers.append(linear[i])
            else:
                self.layers.append(linear[i])
                self.layers.append(activation)
                if (dropout!=0): self.layers.append(torch.nn.Dropout(dropout))
        self.layers = torch.nn.Sequential(*self.layers)
        
        self.softmax = torch.nn.LogSoftmax(dim = 1)
        
    def forward(self, X):          
        X = self.layers(X)
        X = self.softmax(X)
        return X  

In [26]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10,verbose=True):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).to(torch.float32), labels.to(device).to(torch.long)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_train_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        if val_loader is None:
            continue
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device).to(torch.float32), labels.to(device).to(torch.long)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / len(val_loader)
        val_accuracy = correct_val / total_val
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

    # Plotting
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.show()

In [ ]:
Model = MyMLP(layers = [24,64,32,12], dropout=0).to(device)
Model = Model.to(device)
print(Model)
optimizer = torch.optim.Adam(Model.parameters(), lr=0.01)
train_model(Model,torch.nn.NLLLoss(), optimizer,Noise_0_era_train_loader, Noise_0_era_test_loader,num_epochs=200)

In [12]:
len(df)

62400

# train Experts

In [24]:
def train_experts(no_of_experts):
    len_of_individual_train=len(df)//no_of_experts
    start=[i*len_of_individual_train for i in range(no_of_experts)]
    for i in range(len(start)):
        if i==len(start)-1:
            train_loader=customDS.generate_batches_with_labels(start[i],len(df))
        else:
            train_loader=customDS.generate_batches_with_labels(start[i],start[i+1])
        Model = MyMLP(layers = [24,64,32,5], dropout=0).to(device)
        Model = Model.to(device)
        optimizer = torch.optim.Adam(Model.parameters(), lr=0.001)
        train_model(Model,torch.nn.NLLLoss(), optimizer,train_loader, None,num_epochs=200)
        PATH="Experts/expert_no_"+str(i+1)
        torch.save(Model, PATH)
    print("EVERYTHING COMPLETED SUCCESSFULLY")

In [27]:
train_experts(5)

KeyboardInterrupt: 